# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Smith  ->  M. Smith  |  ['M. Smith']
S. Ghosh  ->  S. Ghosh  |  ['S. Ghosh']
S. Ghosh  ->  S. Ghosh  |  ['S. Ghosh']
A. Gould  ->  A. Gould  |  ['A. Gould']


R. Launhardt  ->  R. Launhardt  |  ['R. Launhardt']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']
A. Hughes  ->  A. Hughes  |  ['A. Hughes']


Arxiv has 151 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2309.00671


extracting tarball to tmp_2309.00671...

 done.
Retrieving document from  https://arxiv.org/e-print/2309.00973


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2309.00671/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'table_clusters' from 'tmp_2309.00671/table_clusters.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'icl_redshift_authors' from 'tmp_2309.00671/icl_redshift_authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

✔ → 111055:\section{The luminosity of the central star}
  ↳ 113273:\section{Proper motion of HH 494} \label{sec:app:hh494}
✔ → 113273:\section{Proper motion of HH 494} \label{sec:app:hh494}
  ↳ 116915:end


R. Launhardt  ->  R. Launhardt  |  ['R. Launhardt']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']


Found 91 bibliographic references in tmp_2309.01629/cb26_pap2_v2023Aug31_acc.bbl.
Retrieving document from  https://arxiv.org/e-print/2309.01724


extracting tarball to tmp_2309.01724...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.01629-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.01629) | **A resolved rotating disk wind from a young T Tauri star in the Bok  globule CB26**  |
|| <mark>R. Launhardt</mark>, et al. -- incl., <mark>D. Semenov</mark> |
|*Appeared on*| *2023-09-06*|
|*Comments*| *Accepted by A&A, 25 pages, 19 figures*|
|**Abstract**| The disk-outflow connection plays a key role in extracting excess angular momentum from a forming protostar. We have previously reported the discovery of a small molecular outflow from the edge-on T Tauri star in the Bok globule CB26 that shows a peculiar velocity pattern, reminiscent of an outflow that corotates with the disk. We report new, high-resolution mm-interferometric observations of CB26 with the aim of revealing the morphology and kinematics of the outflow at the disk-outflow interface. The IRAM PdBI was used to observe CO(2-1) at 1.3mm with a resolution of 0.5". Using a physical model of the disk, which was derived from the dust emission, we employed chemo-dynamical modeling combined with line radiative transfer to constrain kinematic parameters and to construct a model of the CO emission from the disk that allowed us to separate the emission of the disk from that of the outflow. Our observations confirm the disk-wind nature of the rotating molecular outflow from CB26. The new high-resolution data reveal an X-shaped morphology of the CO emission close to the disk, and vertical streaks extending from the disk surface with a small half-opening angle of ~7deg, which can be traced out to vertical heights of ~500au. We interpret this emission as the combination of the disk atmosphere and a well-collimated disk wind, which we trace down to vertical heights of 40au, where it is launched from the surface of the flared disk at radii of 20-45au. The observed CO outflow has a total momentum flux of 1e-5 Msun km/s/yr, which is nearly three orders of magnitude larger than the maximum thrust that can be provided by the luminosity of the central star. We conclude that photoevaporation cannot be the main driving mechanism for this outflow, but it must be predominantly an MHD disk wind. It is thus far the best-resolved rotating disk wind observed to be launched from a circumstellar disk. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.00671-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.00671) | **Dark Energy Survey Year 6 Results: Intra-Cluster Light from Redshift 0.2  to 0.5**  |
|| Y. Zhang, et al. -- incl., <mark>M. Smith</mark> |
|*Appeared on*| *2023-09-06*|
|*Comments*| *Submitted to MNRAS*|
|**Abstract**| Using the full six years of imaging data from the Dark Energy Survey, we study the surface brightness profiles of galaxy cluster central galaxies and intra-cluster light. We apply a ``stacking'' method to over four thousand galaxy clusters identified by the redMaPPer cluster finding algorithm in the redshift range of 0.2 to 0.5. This yields high signal-to-noise radial profile measurements of the central galaxy and intra-cluster light out to 1 Mpc from the cluster center. Using redMaPPer richness as a cluster mass indicator, we find that the intra-cluster light brightness has a strong mass dependence throughout the 0.2 to 0.5 redshift range, and the dependence grows stronger at a larger radius. In terms of redshift evolution, we find some evidence that the central galaxy, as well as the diffuse light within the transition region between the cluster central galaxy and intra-cluster light within 80 kpc from the center, may be growing over time. At larger radii, more than 80 kpc away from the cluster center, we do not find evidence of additional redshift evolution beyond the cluster mass dependence, which is consistent with the findings from the IllustrisTNG hydrodynamic simulation. We speculate that the major driver of intra-cluster light growth, especially at large radii, is associated with cluster mass growth. Finally, we find that the color of the cluster central galaxy and intra-cluster light displays a radial gradient that becomes bluer at a larger radius, which is consistent with a stellar stripping and disruption origin of intra-cluster light as suggested by simulation studies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.00973-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.00973) | **Evading no-go for PBH formation and production of SIGWs using Multiple  Sharp Transitions in EFT of single field inflation**  |
|| G. Bhattacharya, et al. -- incl., <mark>S. Ghosh</mark> |
|*Appeared on*| *2023-09-06*|
|*Comments*| *14 pages, 3 figures, Comments are welcome*|
|**Abstract**| Deploying \textit{multiple sharp transitions} (MSTs) under a unified framework, we investigate the formation of Primordial Black Holes (PBHs) and the production of Scalar Induced Gravitational Waves (SIGWs) by incorporating one-loop corrected renormalized-resummed scalar power spectrum. With effective sound speed parameter, $1 \leq c_s \leq 1.17$, the direct consequence is the generation of PBH masses spanning $M_{\rm PBH}\sim{\cal O}(10^{-31}M_{\odot}- 10^{4}M_{\odot})$, thus evading well known \textit{No-go theorem} on PBH mass. Our results align coherently with the extensive NANOGrav 15-year data and the sensitivities outlined by other terrestrial and space-based experiments (e.g.: LISA, HLVK, BBO, HLV(O3), etc.). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.01084-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.01084) | **Rotational Variability and Detection of Superflares in a Young Brown  Dwarf by TESS**  |
|| R. Kumbhakar, et al. -- incl., <mark>S. Ghosh</mark> |
|*Appeared on*| *2023-09-06*|
|*Comments*| *10 pages, 5 figures*|
|**Abstract**| We present a comprehensive analysis of a Transiting Exoplanet Survey Satellite (TESS) high-quality light curve for a young brown dwarf, MHO~4 having spectral type M7.0, in the Taurus star-forming region. We investigate the rotation periods and characterize the BD's dynamic atmosphere and surface features. We present light curve analysis of MHO~4, and estimate the rotation period to be around 2.224~d. Remarkably, MHO~4 exhibits two significant flaring events. Furthermore, we also estimated bolometric flare energies to be within the energy range of $10^{34}$ to $10^{35}$ erg, which sits in the superflare category. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.01724-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.01724) | **Neural network-based emulation of interstellar medium models**  |
|| P. Palud, et al. -- incl., <mark>A. Hughes</mark> |
|*Appeared on*| *2023-09-06*|
|*Comments*| **|
|**Abstract**| The interpretation of observations of atomic and molecular tracers in the galactic and extragalactic interstellar medium (ISM) requires comparisons with state-of-the-art astrophysical models to infer some physical conditions. Usually, ISM models are too time-consuming for such inference procedures, as they call for numerous model evaluations. As a result, they are often replaced by an interpolation of a grid of precomputed models. We propose a new general method to derive faster, lighter, and more accurate approximations of the model from a grid of precomputed models. These emulators are defined with artificial neural networks (ANNs) designed and trained to address the specificities inherent in ISM models. Indeed, such models often predict many observables (e.g., line intensities) from just a few input physical parameters and can yield outliers due to numerical instabilities or physical bistabilities. We propose applying five strategies to address these characteristics: 1) an outlier removal procedure; 2) a clustering method that yields homogeneous subsets of lines that are simpler to predict with different ANNs; 3) a dimension reduction technique that enables to adequately size the network architecture; 4) the physical inputs are augmented with a polynomial transform to ease the learning of nonlinearities; and 5) a dense architecture to ease the learning of simple relations. We compare the proposed ANNs with standard classes of interpolation methods to emulate the Meudon PDR code, a representative ISM numerical model. Combinations of the proposed strategies outperform all interpolation methods by a factor of 2 on the average error, reaching 4.5% on the Meudon PDR code. These networks are also 1000 times faster than accurate interpolation methods and require ten to forty times less memory. This work will enable efficient inferences on wide-field multiline observations of the ISM. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.01280-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.01280) | **KMT-2021-BLG-1547Lb: Giant microlensing planet detected through a signal  deformed by source binarity**  |
|| C. Han, et al. -- incl., <mark>A. Gould</mark> |
|*Appeared on*| *2023-09-06*|
|*Comments*| *9 pages, 4 tables, 7 figures*|
|**Abstract**| We investigate the previous microlensing data collected by the KMTNet survey in search of anomalous events for which no precise interpretations of the anomalies have been suggested. From this investigation, we find that the anomaly in the lensing light curve of the event KMT-2021-BLG-1547 is approximately described by a binary-lens (2L1S) model with a lens possessing a giant planet, but the model leaves unexplained residuals. We investigate the origin of the residuals by testing more sophisticated models that include either an extra lens component (3L1S model) or an extra source star (2L2S model) to the 2L1S configuration of the lens system. From these analyses, we find that the residuals from the 2L1S model originate from the existence of a faint companion to the source. The 2L2S solution substantially reduces the residuals and improves the model fit by $\Delta\chi^2=67.1$ with respect to the 2L1S solution. The 3L1S solution also improves the fit, but its fit is worse than that of the 2L2S solution by $\Delta\chi^2=24.7$. According to the 2L2S solution, the lens of the event is a planetary system with planet and host masses $(M_{\rm p}/M_{\rm J}, M_{\rm h}/M_\odot)=\left( 1.47^{+0.64}_{-0.77}, 0.72^{+0.32}_{-0.38}\right)$ lying at a distance $\D_{\rm L} =5.07^{+0.98}_{-1.50}$~kpc, and the source is a binary composed of a subgiant primary of a late G or an early K spectral type and a main-sequence companion of a K spectral type. The event demonstrates the need of sophisticated modeling for unexplained anomalies for the construction of a complete microlensing planet sample. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2309.01629.md
    + _build/html/tmp_2309.01629/./Figures/fig_cb26_all_moments01_pap_new1_2023Jul19.png
    + _build/html/tmp_2309.01629/./Figures/fig_cb26_all_moments01_pap_new2_2023Jul19.png
    + _build/html/tmp_2309.01629/./Figures/fig_cb26_12co_chanmap_oct2014_obs.png
    + _build/html/tmp_2309.01629/./Figures/fig_cb26_overview_2022Nov.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\micron}{\upmum}$
$\newcommand{\msol}{\mbox{M_\odot}}$
$\newcommand{\mjup}{\mbox{M_{\rm Jup}}}$
$\newcommand{\rsol}{\mbox{R_\odot}}$
$\newcommand{\lsol}{\mbox{L_\odot}}$
$\newcommand{\point}{\cdot}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\HII}{H {\small{II}} }$
$\newcommand{\accrate}{M_\sun{\rm yr}^{-1}}$
$\newcommand{\mjyb}{{\rm mJy beam}^{-1}}$
$\newcommand{\mjyp}{{\rm mJy pixel}^{-1}}$
$\newcommand{\mujyb}{{\rm{\mu}Jy beam}^{-1}}$
$\newcommand{\mujyp}{{\rm{\mu}Jy pixel}^{-1}}$
$\newcommand{\Mjysr}{{\rm MJy sr}^{-1}}$
$\newcommand{\Mjyb}{{\rm MJy beam}^{-1}}$
$\newcommand{\cmcub}{cm^{-3}}$
$\newcommand{\jyb}{{\rm Jy beam}^{-1}}$
$\newcommand{\mjybkms}{{\rm mJy beam}^{-1}{\rm km s}^{-1}}$
$\newcommand{\jybkms}{{\rm Jy beam}^{-1}{\rm km s}^{-1}}$
$\newcommand{\dotsec}{\rlap.{"}}$
$\newcommand{\dotmin}{\rlap.{'}}$
$\newcommand{\dotdeg}{\rlap.{^\circ}}$
$\newcommand{\dd}{  {\rm d}}$
$\newcommand{\env}{{\rm env}}$
$\newcommand{\disk}{{\rm disk}}$
$\newcommand{\rcyl}{r_{\rm cyl}}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\K}{\rm K}$
$\newcommand{\nm}{  \rm nm}$
$\newcommand{\mm}{  \rm mm}$
$\newcommand{\cm}{  \rm cm}$
$\newcommand{\mum}{  \rm \mu m}$
$\newcommand{\msun}{M_\odot}$
$\newcommand{\lsun}{L_\odot}$
$\newcommand{\rsun}{R_\odot}$
$\newcommand{\au}{\rm au}$
$\newcommand{\Jy}{\rm Jy}$
$\newcommand{\beam}{\rm beam}$
$\newcommand{\asp}{\mbox{.\!\!^{\prime\prime}}}$
$\newcommand{\grd}{\mbox{^{\circ}}}$
$\newcommand{\nthp}{\mbox{N_2H^+}}$
$\newcommand{\htdp}{\mbox{H_2D^+}}$
$\newcommand{\ntdp}{\mbox{N_2D^+}}$
$\newcommand{\nht}{\mbox{NH_3}}$
$\newcommand{\hcop}{\mbox{HCO^+}}$
$\newcommand{\htcop}{\mbox{H^{13}CO^+}}$
$\newcommand{\kmspc}{\mbox{km s^{-1} pc^{-1}}}$
$\newcommand{\kms}{\mbox{km s^{-1}}}$
$\newcommand{\htp}{\mbox{H_3^+}}$
$\newcommand{\Ka}{\mbox{K_{\rm a}}}$
$\newcommand{\rl}[1]{{\leavevmode\color{magenta} #1}}$
$\newcommand{\zcp}[1]{{\leavevmode\color{cyan} #1}}$</div>



<div id="title">

# A resolved rotating disk wind from a young T Tauri star in the Bok globule CB 26      $\thanks{Based on observations carried out with the IRAM Plateau de Bure Interferometer (PdBI, now NOEMA), the Owens Valley millimeter-wave array (OVRO), and the Submillimeter Array (SMA).}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.01629-b31b1b.svg)](https://arxiv.org/abs/2309.01629)<mark>Appeared on: 2023-09-06</mark> -  _Accepted by A&A, 25 pages, 19 figures_

</div>
<div id="authors">

<mark>R. Launhardt</mark>, et al. -- incl., <mark>D. Semenov</mark>

</div>
<div id="abstract">

**Abstract:** The disk-outflow connection plays a key role in extracting excess angular momentum from a forming protostar. Although indications of jet rotation have been reported for a few objects, observational constraints of outflow rotation are still very scarce. We have previously reported the discovery of a small collimated molecular outflow from the  edge-on T Tauri star -- disk system in the Bok globule CB 26 that shows a peculiar velocity pattern, reminiscent of an outflow that corotates with the Keplerian disk. However, we could not ultimately exclude possible alternative explanations for the origin of the observed velocity field. We report new, high angular resolution millimeter-interferometric observations of CB 26 with the aim of revealing the morphology and kinematics of the outflow at the disk -- outflow interface to unambiguously discriminate between the possible alternative explanations for the observed peculiar velocity pattern. The IRAM PdBI array and the 30 m telescope were used to observe HCO $^{+}$ (1--0) and H $^{13}$ CO $^{+}$ (1--0) at 3.3 mm and $\mbox{$^{12}$CO(2--1)}$ at 1.3 mm in three configurations plus zerospacing, resulting in spectral line maps with angular resolutions of 3 $\farcs$ 5 and 0 $\farcs$ 5, respectively. The SMA was used to observe the HCO $^{+}$ (3--2) line at 1.1 mm with an angular resolution of 1 $\farcs$ 35. Additional earlier observations of $^{13}$ CO(1--0) at 2.7 mm with an angular resolution of 1 $\farcs$ 0, obtained with OVRO, are also used for the analysis. Using a physical model of the disk, which was derived from the dust continuum emission, we employed chemo-dynamical modeling combined with line radiative transfer calculations to constrain kinematic parameters of the system and to construct a model of the CO emission from the disk that allowed us to separate the emission of the disk from that of the outflow. Our observations confirm the disk-wind nature of the rotating molecular outflow from $\mbox{CB 26 - YSO 1}$ . The new high-resolution data reveal an X-shaped morphology of the CO emission close to the disk, and vertical streaks extending from the disk surface with a small half-opening angle of $\approx$ 7◦, which can be traced out to vertical heights of $\approx$ 500 au. We interpret this emission as the combination of the disk atmosphere and a well-collimated disk wind, of which we mainly see the outer walls of the outflow cone. The decomposition of this emission into a contribution from the disk atmosphere and the disk wind allowed us to trace the disk wind down to vertical heights of $\approx$ 40 au, where it is launched from the surface of the flared disk at radii of $R_{\rm L}\approx$ 20 -- 45 au.The disk wind is rotating with the same orientation and speed as the Keplerian disk and the velocity structure of the cone walls along the flow is consistent with angular momentum conservation.The observed CO outflow has a total gas mass of $\approx10^{-3}$ $\msol$ , a dynamical age of $\mbox{$\tau_{\rm dyn}\approx740$ yr}$ , and a total momentum flux of $\dot{P}_{\rm CO}\approx 1.0\times10^{-5}$ $\msol$ $\kms$ yr $^{-1}$ , which is nearly three orders of magnitude larger than the maximum thrust that can be provided by the luminosity of the central star. We conclude that photoevaporation cannot be the main driving mechanism for this outflow, but it must be predominantly a magnetohydrodynamic (MHD) disk wind. It is thus far the best-resolved rotating disk wind observed to be launched from a circumstellar disk in Keplerian rotation around a low-mass young stellar object (YSO), albeit also the one with the largest launch radius. It confirms the observed trend that disk winds from Class I YSOs with transitional disks have much larger launch radii than jets ejected from Class 0 protostars.

</div>

<div id="div_fig1">

<img src="tmp_2309.01629/./Figures/fig_cb26_all_moments01_pap_new1_2023Jul19.png" alt="Fig5.1" width="50%"/><img src="tmp_2309.01629/./Figures/fig_cb26_all_moments01_pap_new2_2023Jul19.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** 
Line spectra of CB 26, integrated over $\approx$1.5 beams around the disk center (top row), integrated intensity maps (middle row), and mean velocity fields (1$^{\rm st}$ moment maps, bottom row). From left to right:  
$^{12}$CO (2--1) (0.5 to 4.8 and 7.0 to 12.0 $\kms$),
$^{13}$CO (1--0) (3.4 to 4.9 and 6.2 to 7.7 $\kms$),
HCO$^+$(3--2) (1 to 10 $\kms$), and
HCO$^+$(1--0) and H$^{13}$CO$^+$(1--0) in blue (2.0 to 5.0 and 6.3 to 9.0 $\kms$).
Dashed vertical lines in the spectra (top) indicate the velocity range within which the spectra are most strongly affected by resolved-out emission and self-absorption from the extended envelope (see Figs. \ref{fig_chanmap_12co_obs} through \ref{fig_chanmap_hco10}). Total intensity contours in the maps start at 3 $\sigma$. Overlaid as red contours in the middle row at 2.5, 7.5, and 15 mJy/beam is the 230 GHz dust continuum emission from the disk. Synthesized FWHM beam sizes are shown as gray ellipses in the lower right corners (dark gray: line, light gray: continuum). The bottom panels show the respective 1$^{\rm st}$ moment maps with contours of the total intensity overlaid. A black triangle marks the location of the central star (and center of the disk). We note that, except for HCO$^+$(3--2), the envelope-dominated central velocity channels were masked out before the moment maps where generated. (*fig_intmaps1*)

</div>
<div id="div_fig2">

<img src="tmp_2309.01629/./Figures/fig_cb26_12co_chanmap_oct2014_obs.png" alt="Fig7" width="100%"/>

**Figure 7. -** $^{12}$CO (2--1) channel maps of CB 26, obtained with PdBI in 2005 and 2009, rotated counterclockwise by $32◦$.
Contour levels start at 15 mJy/beam (2 $\sigma$ r.m.s., see Table \ref{tab-obs}).
Red contours mark the 1.2 mm dust continuum emission from the disk (3 and 15 mJy/beam.
The reference position is $\alpha_{2000} = 04^h59^m50.74^s$,
                          $\delta_{2000} = 52^{\circ}04^{\prime}43.80^{\prime\prime}$.
The $^{12}$CO synthesized beam size is indicated as the gray ellipse in the lower right
corner of the first channel map. The channel number and mean $\Delta v = v_{\rm LSR}-5.95$(in $\kms$) are indicated in the top left and right corners of the maps, respectively.
Maps in the central velocity channels between $\Delta v\approx\pm1.2$\kms are corrupted by
resolved-out emission and self-absorption from the extended envelope.
 (*fig_chanmap_12co_obs*)

</div>
<div id="div_fig3">

<img src="tmp_2309.01629/./Figures/fig_cb26_overview_2022Nov.png" alt="Fig4" width="100%"/>

**Figure 4. -** 
 Overview of the CB 26 region. The left panel shows a wide-field optical "true-color" image, which is based on H$\alpha$(blue), [SII](green), and I--band (red) images  ([Stecklum, Launhardt and Fischer 2004]()) . The globules CB 25 and CB 26 as well as Herbig-Haro object HH 494 are marked in the image. The zoomed-in right panel shows a NIR J-band image of the bipolar reflection nebula (color, 0$\asp$6 resolution), overlaid with contours of the 1.3 mm dust continuum emission from the disk (white contours at 3, 7.5, and 15 mJy/beam). Green contours show the integrated $^{12}$CO(2--1) emission (0.5 to 12 $\kms$) from the bipolar molecular outflow as presented in [Launhardt, Pavlyuchenkov and Gueth (2009)](). The red and blue arrows indicate the large-scale outflow orientation. Beam sizes are shown in respective colors in the lower right corner. The white dashed square marks the image section shown in Fig. \ref{fig:contim}. The reference position is  $04^h59^m50.74^s, 52^{\circ}04^{\prime}43.80^{\prime\prime}$(J2000).
 (*fig-overview*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.01629"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

261  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
